# Variability analysis as a subset of model building

The same machinery that is used to iterate over component parameters to build models can be used to study the effect of variability on device performance.

## Lithographic parameters

`LithoParameter` parameters have a parametrizable `transformation` attribute that can be used to modify the Component geometry prior to simulation in more complex way than simply changing its calling arguments.

## Corner analysis

For convenience, the model builder can also iterate over only the `min`, `max`, and `nominal` values of all trainable_parameters by using the `types=corners` instead of the default `types=arange` argument of `Model.get_model_input_output(type="corners")`.

## Directional coupler example

Consider a directional coupler component which is modeled through a generic `MeepFDTDModel`. The only difference between this and the `FemwellWaveguideModel` from last notebook is how the simulation is defined: everything else involving iteration over parameters, multiprocessing, and model fitting, is identical. This makes model building easily extensible to new simulators.

In [ ]:
import gdsfactory as gf
from gdsfactory.simulation.sax.parameter import NamedParameter
from gdsfactory.technology import LayerStack
from gdsfactory.pdk import _ACTIVE_PDK, get_layer_stack


# gdsfactory layerstack
filtered_layerstack = LayerStack(
    layers={
        k: get_layer_stack().layers[k]
        for k in (
            "slab90",
            "core",
            "box",
            "clad",
        )
    }
)

# trainable component function, choosing which parameters to fix and which to consider for the model
def trainable_coupler(parameters):
    return gf.components.coupler_full(
        coupling_length=parameters["coupling_length"],
        gap=parameters["gap"],
        dw=0.0,
    )

parameters = {"gap": 0.4, "coupling_length": 10}
c = trainable_coupler(parameters)
c.plot()

When defining the model, we add the LithoParameter `erosion_magnitude`. For all models, a `TransformParameter` which if set, will offset the provided component prior to simulation, emulating erosion (when <1), nominal behaviour (when 1) and dilation (when >1). This morphological transformation is currently global; more advanced spatially-correlated filters are an obvious next step.

In [ ]:
from gdsfactory.simulation.sax.meep_FDTD_model import MeepFDTDModel

# Simulation settings
port_symmetries_coupler = {
    "o1@0,o1@0": ["o2@0,o2@0", "o3@0,o3@0", "o4@0,o4@0"],
    "o2@0,o1@0": ["o1@0,o2@0", "o3@0,o4@0", "o4@0,o3@0"],
    "o3@0,o1@0": ["o1@0,o3@0", "o2@0,o4@0", "o4@0,o2@0"],
    "o4@0,o1@0": ["o1@0,o4@0", "o2@0,o3@0", "o3@0,o2@0"],
}

sim_settings = dict(
    resolution=10,
    xmargin=1.0,
    ymargin=1.0,
    is_3d=False,
    port_source_names=["o1"],
    port_symmetries=port_symmetries_coupler,
    run=True,
    overwrite=False,
    layer_stack=filtered_layerstack,
    z=0.1,
)


coupler_model = MeepFDTDModel(trainable_component=trainable_coupler,
    layerstack=filtered_layerstack,
    simulation_settings={
        "sim_settings": sim_settings,
    },
    trainable_parameters={
        "coupling_length": NamedParameter(
            min_value=10, max_value=10, nominal_value=10, step=10
        ),
        "gap": NamedParameter(
            min_value=0.2, max_value=0.2, nominal_value=0.2, step=0.3
        ),
    },
    non_trainable_parameters={
        "wavelength": NamedParameter(
            min_value=1.54, max_value=1.56, nominal_value=1.55, step=0.01
        ),
    },
    num_modes=1,
)

In [ ]:
input_vectors, output_vectors = coupler_model.get_model_input_output(type="arange")